In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sc
import scipy.sparse as sp
import itertools

from hypernetx.extras import lesmis as lm
import matplotlib.pyplot as plt
import pygenstability as pgs
from pygenstability import plotting

In [ ]:
import hypernetx as hnx

# Construct random walk matrix



In [ ]:
def get_laplacian(H):
    
    """ Construct Laplacian for HyperGraph H
    
    Arguments
    H : Hypernetx hypergraph object
    
    """

    incidence = H.incidence_matrix().toarray()
    
    # hyperedge adjacency matrix
    C = np.matmul(incidence.T,incidence)
    A = np.matmul(incidence,incidence.T)

    R = np.matmul(incidence, np.matmul(np.diag(np.diag(C)),incidence.T))

    # defining transition matrix
    L = R - A
    np.fill_diagonal(L,0)
    np.fill_diagonal(L,-np.sum(L,axis=0)[:,None])
    L
    
    return L
  
def get_transition_matrix(H):
    
    """ Construct Laplacian for HyperGraph H
    
    Arguments
    H : Hypernetx hypergraph object
    
    """

    incidence = H.incidence_matrix().toarray()
    
    # hyperedge adjacency matrix
    C = np.matmul(incidence.T,incidence)
    A = np.matmul(incidence,incidence.T)

    R = np.matmul(incidence, np.matmul(np.diag(np.diag(C)),incidence.T))

    # defining transition matrix
    T = R - A
    np.fill_diagonal(T,0)
    T = T/np.sum(T,axis=0)[:,None]
    
    return T

def get_adjacency(H):
    
    """ Construct Laplacian for HyperGraph H
    
    Arguments
    H : Hypernetx hypergraph object
    
    """

    incidence = H.incidence_matrix().toarray()
    
    # hyperedge adjacency matrix
    C = np.matmul(incidence.T,incidence)
    A = np.matmul(incidence,incidence.T)

    R = np.matmul(incidence, np.matmul(np.diag(np.diag(C)),incidence.T))

    # defining transition matrix
    adj = R - A
    np.fill_diagonal(adj,0)

    
    return adj

# Basic Example 

In [ ]:
scenes = {
    0: ('a', 'b', 'c'),
    1: ('c', 'd',),
    2: ('a','b',)
    
}

H = hnx.Hypergraph(scenes)

In [ ]:
hnx.draw(H)

In [ ]:
HD = H.dual()
hnx.draw(HD)
H.nodes

In [ ]:
L = get_laplacian(H)
L

In [ ]:
T = get_transition_matrix(H)
T

In [ ]:
A = get_adjacency(H)
A

In [ ]:
# non-hypergraph adjacency matrix
incidence = H.incidence_matrix().toarray()    
A_ = np.matmul(incidence,incidence.T)
np.fill_diagonal(A_,0)
A_

# Application to subgraph of Les Mis

In [ ]:
scenes = {
    0: ('FN', 'TH'),
    1: ('TH', 'JV'),
    2: ('BM', 'FN', 'JA'),
    3: ('JV', 'JU', 'CH', 'BM'),
    4: ('JU', 'CH', 'BR', 'CN', 'CC', 'JV', 'BM'),
    5: ('TH', 'GP'),
    6: ('GP', 'MP'),
    7: ('MA', 'GP')
}

H = hnx.Hypergraph(scenes)

In [ ]:
hnx.draw(H)

In [ ]:
L = get_laplacian(H)
L

In [ ]:
T = get_transition_matrix(H)
T.round(1)

In [ ]:
adj = get_adjacency(H)
adj

In [ ]:
adj = sp.csr_matrix(adj)


# Markov Stability on Les Mis

In [ ]:
volumes = lm.volumes;
### List of Characters
names = lm.df_names.set_index('Symbol');
scenes = lm.df_scenes


In [ ]:
### Construct the edges as a dictionary named by the name of the Volume
volume_edges = dict()
for v in range(1,6):
    volume_edges[v] = set(scenes.loc[scenes.Volume == v]['Characters'])
    
### Construct a hypergraph made up of volume_edges
H = hnx.Hypergraph(volume_edges,name='Volumes')
for node in H.nodes:
    H.nodes[node].name = names.loc[node]['FullName']
    H.nodes[node].description = names.loc[node]['Description']

In [ ]:
### Visualize the Hypergraph
def noborder(width=10,height=10):
    fig = plt.figure(figsize=[width,height])
    ax = plt.gca()
    ax.axis('off')
noborder()
hnx.draw(H)


# construct projected matrix (no hyperedges)
incidence = H.incidence_matrix().toarray()    
A_ = np.matmul(incidence,incidence.T)
np.fill_diagonal(A_,0)
A_ = sp.csr_matrix(A_)

# construct network object just for plotting
g = nx.Graph(A_)
pos = nx.spring_layout(g, weight=None, scale=1)
for u in g:
    g.nodes[u]["pos"] = pos[u]


# Synthetic Example 

In [ ]:
edges = {
    0: ('1','2','3','4'),
    1: ('5','6','7','8'),
    2: ('9','10','11','12'),
    3: ('13','14','15','16'), 
    4: ('2','5'),
    5: ('4','7'),
    6: ('2','7'),
    7: ('4','5'),
    8: ('7','13'),
    9: ('8','14'),
    10: ('7','14'),
    11: ('13','8'),
    12: ('3','9'),
    13: ('4','10'),
    14: ('3','10'),
    15: ('4','9'),
    16: ('10','13'),
    17: ('12','15'),
    18: ('10','15'),
    19: ('12','13'),
    20: ('4','13'),
    21: ('7','10')
}

H = hnx.Hypergraph(edges)
hnx.draw(H)


# adjacency matrix constructed using Carletti method
graph = sp.csr_matrix(get_adjacency(H))

# construct projected matrix (no hyperedges)
incidence = H.incidence_matrix().toarray()    
A_ = np.matmul(incidence,incidence.T)
np.fill_diagonal(A_,0)
A_ = sp.csr_matrix(A_)

# construct network object just for plotting
g = nx.Graph(A_)
pos = nx.spring_layout(g, weight=None, scale=1)
for u in g:
    g.nodes[u]["pos"] = pos[u]

In [ ]:

results_hypergraph = pgs.run(graph, min_time=-2, max_time=1, n_time=50, constructor='continuous_combinatorial')
plotting.plot_scan(results_hypergraph)
plt.figure()
plotting.plot_single_community(g, results_hypergraph,30)

In [ ]:
# standard adjacency matrix

results_projected = pgs.run(A_, min_time=-2, max_time=1, n_time=50, constructor='continuous_combinatorial')
plotting.plot_scan(results_projected)
plt.figure()
plotting.plot_single_community(g, results_projected,30)

# Synthetic Example 2



In [ ]:
edges = {
    0: ('2','3','7','8'),
    1: ('4','5','9','10'),
    2: ('1','2'),
    3: ('6','7'), 
    4: ('1','7'),
    5: ('6','2'),
    6: ('3','4'),
    7: ('8','9'),
    8: ('3','9'),
    9: ('4','8'),
    10: ('1','6')
}

H = hnx.Hypergraph(edges)
hnx.draw(H)


# adjacency matrix constructed using Carletti method
graph = sp.csr_matrix(get_adjacency(H))

# construct projected matrix (no hyperedges)
incidence = H.incidence_matrix().toarray()    
A_ = np.matmul(incidence,incidence.T)
np.fill_diagonal(A_,0)
A_ = sp.csr_matrix(A_)

# construct network object just for plotting
g = nx.Graph(A_)
pos = nx.spring_layout(g, weight=None, scale=1)
for u in g:
    g.nodes[u]["pos"] = pos[u]



In [ ]:
results_hypergraph = pgs.run(graph, min_time=-2, max_time=1, n_time=50, constructor='continuous_combinatorial')
plotting.plot_scan(results_hypergraph)
plt.figure()
plotting.plot_single_community(g, results_hypergraph,30)

In [ ]:
# standard adjacency matrix

results_projected = pgs.run(A_, min_time=-2, max_time=1, n_time=50, constructor='continuous_combinatorial')
plotting.plot_scan(results_projected)
plt.figure()
plotting.plot_single_community(g, results_projected,30)

# Stochastic block model



In [ ]:
from sklearn.metrics import mutual_info_score



def joint_entropy(x,y):
    """
    Calculate the entropy of a variable, or joint entropy of several variables.
    Parameters
    ----------
    x : array or list
    y : array or list

    """
    n_instances = len(x)
    H = 0
    X = [np.array(x),np.array(y)]
    for classes in itertools.product(*[set(x) for x in X]):
        v = np.array([True] * n_instances)
        for predictions, c in zip(X, classes):
            v = np.logical_and(v, predictions == c)
        p = np.mean(v)
        H += -p * np.log2(p) if p > 0 else 0
    return H

def entropy(labels):
    """Calculates the entropy for a labeling.
    Parameters
    ----------
    labels : int array, shape = [n_samples]
        The labels
    Notes
    -----
    The logarithm used is the natural logarithm (base-e).
    """
    if len(labels) == 0:
        return 1.0
    label_idx = np.unique(labels, return_inverse=True)[1]
    pi = np.bincount(label_idx).astype(np.float64)
    pi = pi[pi > 0]
    pi_sum = np.sum(pi)
    # log(a / b) should be calculated as log(a) - log(b) for
    # possible loss of precision
    return -np.sum((pi / pi_sum) * (np.log(pi) - np.log(pi_sum)))

def vi_score(partition_1,partition_2):

    MI = mutual_info_score(
            partition_1,
            partition_2,
        )
    Ex = entropy(partition_1)
    Ey = entropy(partition_2)
    JE = Ex + Ey - MI

    return (JE - MI) / JE


def construct_hypergraph_sbm(g):

    cliques = list(nx.find_cliques(g))
    cli = cliques

        #cli = [edge for edge in cli if len(edge)>2]

    cli.sort(key=len)
    cli.reverse()

    h_block_edges = []


    for edge in cli:
        blocks = [g.nodes[node]['block'] for node in edge]
        if blocks.count(blocks[0]) == len(blocks):
            h_block_edges.append(edge)


    cli = h_block_edges + [edge for edge in cliques if len(edge)<3]

    # this removal section needs fixing!!!!!
    h_edges = []
    while len(cli)>0:
        edge = cli[0]#cli[np.random.randint(len(cli),size = 1)[0]]
        for e in cli:
            intersect = list(set(e) & set(edge)) 
            if len(intersect)>1:    
                cli.remove(e)

        h_edges.append(tuple(edge))

    a = np.linspace(0, len(h_edges)-1, len(h_edges))
    a = [str(int(x)) for x in a]


    h_edge_list = dict(zip(a, h_edges))
    H = hnx.Hypergraph(h_edge_list)
    return H

In [ ]:

ps = np.linspace(0.1,0.5,10)

sizes = [20,20,20]
p1 = 0.9

vi_scores_projected = []
vi_scores_hypergraph = []

for p2  in ps:
    probs = [[p1, p2, p2], [p2, p1, p2], [p2, p2, p1]]
    g = nx.stochastic_block_model(sizes, probs, seed=0)
    #nx.draw(g)
    H = construct_hypergraph_sbm(g)
    
    # adjacency matrix constructed using Carletti method
    graph = sp.csr_matrix(get_adjacency(H))

    # construct projected matrix (no hyperedges)
    incidence = H.incidence_matrix().toarray()    
    A_ = np.matmul(incidence,incidence.T)
    np.fill_diagonal(A_,0)
    A_ = sp.csr_matrix(A_)
    
    # construct network object just for plottin
        
    gt_partition = [g.nodes[node]['block'] for node in g]
        
    results_hypergraph = pgs.run(graph, min_time=-1, max_time=1.5, n_time=50, constructor='continuous_combinatorial')
    comms = np.array(results_hypergraph['number_of_communities'])
    idx = np.where(comms==3)[0] # take first index
    vi = []
    for i in idx:        
        partition_hypergraph = results_hypergraph['community_id'][i] 
        vi.append(vi_score(partition_hypergraph,gt_partition))
    vi_scores_hypergraph.append(np.median(vi))
    
    
    results_projected = pgs.run(A_, min_time=-1, max_time=1.5, n_time=50, constructor='continuous_combinatorial')
    comms = np.array(results_hypergraph['number_of_communities'])
    idx = np.where(comms==3)[0] # take first index
    vi = []
    for i in idx:        
        partition_projected = results_projected['community_id'][i] 
        vi.append(vi_score(partition_projected,gt_partition))
    vi_scores_projected.append(np.median(vi))


In [ ]:
len(H)

In [ ]:
plt.plot(ps,vi_scores_hypergraph)
plt.plot(ps,vi_scores_projected)

In [ ]:
sizes = [4,4]

p1 = 1
p2 = 0.05
#probs = [[p1, p2, p2], [p2, p1, p2], [p2, p2, p1]]
p = [[p1, p2,],[p2, p1]]

In [ ]:
import random

nodelist = range(0, sum(sizes))
block_range = range(len(sizes))
g = nx.Graph()
block_iter = itertools.combinations_with_replacement(block_range, 2)
    # Split nodelist in a partition (list of sets).
size_cumsum = [sum(sizes[0:x]) for x in range(0, len(sizes) + 1)]
g.graph["partition"] = [
        set(nodelist[size_cumsum[x] : size_cumsum[x + 1]])
        for x in range(0, len(size_cumsum) - 1)
    ]
    # Setup nodes and graph name
for block_id, nodes in enumerate(g.graph["partition"]):
    for node in nodes:
        g.add_node(node, block=block_id)

g.name = "hypergraph_stochastic_block_model"

    # Test for edge existence
parts = g.graph["partition"]
for i, j in block_iter:
        edges = itertools.product(parts[i], parts[j])             

        for e in edges:
            if random.uniform(0, 1) < p[i][j]:
                g.add_edge(*e)  # __safe  
                
nx.draw(g)

In [ ]:
sizes = [4,4]

p1 = 1
p2 = 0.05
#probs = [[p1, p2, p2], [p2, p1, p2], [p2, p2, p1]]
probs = [[p1, p2,],[p2, p1]]


g = nx.stochastic_block_model(sizes, probs, seed=0)


nx.draw(g)



In [ ]:
cliques = list(nx.find_cliques(g))
h_block_edges = []
for edge in cliques:
    blocks = [g.nodes[node]['block'] for node in edge]
    
    if blocks.count(blocks[0]) == len(blocks):
        h_block_edges.append(edge)
        
h_block_edges

In [ ]:
for edge in h_block_edges:
    


In [ ]:

for e in cli:
    intersect = list(set(e) & set(edge))
    print(len(intersect))
    if len(intersect)>1: 
        print(e)
        cli.remove(e)

In [ ]:
print(cli)
edge = cli[0]#cli[np.random.randint(len(cli),size = 1)[0]]
print(edge)
print(len(cli))

cli_c = cli

for e in cli:
    intersect = list(set(e) & set(edge))
    print(len(intersect))
    if len(intersect)>1: 
        print(e)
        cli.remove(e)

        
print(cli)


In [ ]:

h_edges = []
while len(cli)>0:
    edge = cli[0]#cli[np.random.randint(len(cli),size = 1)[0]]
    print(edge)
    for e in cli:
        intersect = list(set(e) & set(edge)) 
        if len(intersect)>1:    
            cli.remove(e)

    h_edges.append(tuple(edge))

a = np.linspace(0, len(h_edges)-1, len(h_edges))
a = [str(int(x)) for x in a]


h_edge_list = dict(zip(a, h_edges))
H = hnx.Hypergraph(h_edge_list)

nx.draw(g)


In [ ]:
h_edges

In [ ]:
nx.number_of_edges(g)